# Dimension-Dependence of MFGS

<span style="font-size:1.2em;">! Run this in **Julia** !</span> 

1. You'll need to install the Julia add on for Jupyter Notebooks (IJulia). Follow guide here: https://datatofish.com/add-julia-to-jupyter/
2. Go to *Kernel* -> *Change Kernel* -> *Julia*

## Packages

In [1]:
include("RCLib.jl") # Include the RC library with all the useful functions
using .RCLib

using LinearAlgebra # For IIL matrix multiplication
using Kronecker
using OrdinaryDiffEq
using DifferentialEquations

## Plotting Packages ##
using CSV # Allows data to be converted to .CSV to use with Python for plotting
using DataFrames # Another useful package for exporting data
using Plots

## Performance Packages ##
using BenchmarkTools

## Variables

### Lorentzian Spectral Density

In [2]:
prm1 = [7.0 5.0 10.0];
prm2 = [1.4 0.5 0.16];
prm3 = [1.4 0.001 10];

In [3]:
ω0x, Γx, αx = prm3*ωL;
ω0y, Γy, αy = prm3*ωL;
ω0z, Γz, αz = prm3*ωL;

### RC-Specific

In [4]:
Ωx = ω0x;
λx = ωL*sqrt(αx/Ωx);
δx = Γx/(2π*Ωx);
nx = 5;

In [5]:
Ωy = ω0y;
λy = ωL*sqrt(αy/Ωy);
δy = Γy/(2π*Ωy);
ny = 5;

In [6]:
Ωz = ω0z;
λz = ωL*sqrt(αz/Ωz);
δz = Γz/(2π*Ωz);
nz = 5;

## Gibbs State

In [7]:
HG(T) = gibbs(HSpG(nx), T);

In [8]:
ρSpG(T)= ptrace(HG(T), nx);

In [9]:
tr(ρSpG(1))

1.0000000000000002

## 1D Mean-Force Gibbs State


In [10]:
Htot1D = HRC1D(nx, λx, Ωx);

In [11]:
ρtot1D(T) = gibbs(Htot1D, T);

In [12]:
ρSpMFGS1D(T) = ptrace(ρtot1D(T), nx);

In [13]:
tr(ρSpMFGS1D(1))

0.9999999999999999

In [14]:
GS = eigvecs(Htot1D)[:,1];

## 2D Mean-Force Gibbs State

In [15]:
Htot2D = HRC2D(nx, nz, λx, λz, Ωx, Ωz);

In [16]:
ρtot2D(T) = gibbs(Htot2D, T);

In [17]:
ρSpMFGS2D(T) = ptrace(ptrace(ρtot2D(T), nz), nx);

In [18]:
tr(ρSpMFGS2D(1))

1.0000000000000002

## 3D Mean-Force Gibbs State

In [19]:
Htot3D = HRC3D(nx, ny, nz, λx, λy, λz, Ωx, Ωy, Ωz);

In [20]:
ρtot3D(T) = gibbs(Htot3D, T);

In [21]:
ρSpMFGS3D(T) = ptrace(ptrace(ptrace(ρtot3D(T), nz), ny), nx);

In [22]:
tr(ρSpMFGS3D(1))

1.0000000000000002

# Plots

In [23]:
T_list = 0.001:1:1000;

In [24]:
expzG(T) = real(tr(ρSpG(T)*sz0)); # Gibbs Sz expectation value
expz1DMFGS(T) = real(tr(ρSpMFGS1D(T)*sz0)); # 1D MFGS Sz expectation value
expz2DMFGS(T) = real(tr(ρSpMFGS2D(T)*sz0)); # 2D MFGS Sz expectation value
expz3DMFGS(T) = real(tr(ρSpMFGS3D(T)*sz0)); # 2D MFGS Sz expectation value

In [27]:
T = [i for i in T_list];
szG = [expzG(i) for i in T_list];
sz1DMFGS = [expz1DMFGS(i) for i in T_list];
sz2DMFGS = [expz2DMFGS(i) for i in T_list];
sz3DMFGS = [expz3DMFGS(i) for i in T_list];
df = DataFrame(hcat(T, szG, sz1DMFGS, sz2DMFGS, sz3DMFGS), :auto);

In [28]:
CSV.write("C://Users//crh222//QuantumSpinDynamics//MFGS_data.csv",  df, header = ["T", "szG", "sz1DMFGS", "sz2DMFGS", "sz3DMFGS"]);